In [1]:
import os
import io
import logging
from kafka import KafkaProducer
from PIL import Image
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, struct
from pyspark.sql.types import ArrayType, FloatType
import segmentation_models_pytorch
import torch
from torchvision import transforms
import cv2
import numpy as np
from pyspark.ml.functions import predict_batch_udf
from pymongo import MongoClient
import gridfs


d:\Anaconda\envs\cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

python_version = 'D:/Anaconda/envs/cuda/python.exe'
os.environ['PYSPARK_PYTHON'] = python_version
os.environ['PYSPARK_DRIVER_PYTHON'] = python_version

In [3]:
# Setup MongoDB
client = MongoClient('localhost', 27017)
db = client['image_db']
collection = db['image_masks']
fs = gridfs.GridFS(db)

In [4]:
# Kafka and Spark configuration
kafka_server = 'localhost:9092'
topic_name = 'RandomImage'
scala_version = '2.12'
spark_version = '3.5.1'
packages = [f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}', 'org.apache.kafka:kafka-clients:3.7.0']

spark = SparkSession.builder.master("local").appName("kafka-example") \
                    .config("spark.driver.memory", "23g") \
                    .config("spark.executor.memory", "23g") \
                    .config("spark.executor.cores", "8") \
                    .config("spark.sql.shuffle.partitions", "1000") \
                    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
                    .config("spark.jars.packages", ",".join(packages)) \
                    .config("spark.network.timeout", "6000s") \
                    .config("spark.executor.heartbeatInterval", "1000s") \
                    .config("spark.streaming.kafka.consumer.poll.ms", "600000") \
                    .config("spark.streaming.backpressure.enabled", "true") \
                    .config("spark.streaming.receiver.maxRate", "50") \
                    .getOrCreate()

In [5]:
MODEL_LOAD_PATH = f'./models/supreme/abhi_sudo_full_2-pretrained_preproc_1-deeplabv3+-ep_8-0.001-30.pt'
SAVE_PATH = f'./predictionsDeeplab'

os.makedirs(SAVE_PATH, exist_ok=True)


# ####### DEEPLAB V3+ CONFIG #######
ENCODER_DEPTH=5
DECODER_CHANNELS=256
BATCH_SIZE= 16
EPOCHS= 100
LR = [1e-3]
ENCODER_NAME= 'resnet50'

CLASSES={'Background':0,'Building-flooded':1,'Building-non-flooded':2,'Road-flooded':3,'Road-non-flooded':4,
         'Water':5,'Tree':6,'Vehicle':7,'Pool':8,'Grass':9}
IMG_DIM= 512

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
model = segmentation_models_pytorch.DeepLabV3Plus(encoder_name=ENCODER_NAME, encoder_depth=ENCODER_DEPTH,
                                              decoder_channels=DECODER_CHANNELS, classes=len(CLASSES))


model = model.to(device)
model.load_state_dict(torch.load(MODEL_LOAD_PATH,map_location=torch.device('cuda')))
# model = model.to("cpu")
model.eval()

DeepLabV3Plus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequentia

In [8]:

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def reverse_transform_mask(inp):
    inp = inp.transpose((1, 2, 0))
    t_mask = np.argmax(inp, axis=2).astype('float32')
    t_mask = cv2.resize(t_mask, dsize=(4000, 3000))
    kernel = np.ones((3, 3), np.uint8)
    t_mask = cv2.erode(t_mask, kernel, iterations=1)
    return t_mask

def transform_image(image_bytes):
    img = Image.open(io.BytesIO(image_bytes))
    img = img.resize((IMG_DIM, IMG_DIM))
    img = np.array(img)
    kernel = np.ones((2, 2), np.uint8)
    img = cv2.bilateralFilter(img, 5, 75, 75)
    img = cv2.erode(cv2.dilate(img, kernel, iterations=2), kernel, iterations=1)
    transformed_img = val_transform(img)
    return transformed_img.numpy().flatten().astype(np.float32)

def predict_batch_fn():
    
    model.eval()

    def predict(inputs: np.ndarray):
        inputs = inputs.reshape(-1, 3, IMG_DIM, IMG_DIM)
        inputs_tensor = torch.from_numpy(inputs).to(device).float()
        with torch.no_grad():
            preds = model(inputs_tensor)
            preds = torch.sigmoid(preds).cpu().numpy()
        # Ensure each prediction is two-dimensional
        predictions_2d = preds.reshape(inputs.shape[0], -1)
        return predictions_2d

    return predict

In [9]:

image_predict_udf = predict_batch_udf(
    predict_batch_fn,
    return_type=ArrayType(FloatType()),
    batch_size=BATCH_SIZE,
    input_tensor_shapes=[[3 * IMG_DIM * IMG_DIM]]
)

In [10]:

def foreach_batch_function(df, epoch_id):
    logger.info(f"Foreach Batch Function called on Epoch ID: {epoch_id}.")
    print(f"Foreach Batch Function called on Epoch ID: {epoch_id}.")
    if df.count() > 0:
        logger.info(f"Processing {df.count()} records in the batch.")
        print(f"Processing {df.count()} records in the batch.")
        pandas_df = df.toPandas()
        transformed_images = []
        keys = []

        for index, row in pandas_df.iterrows():
            transformed_img = transform_image(row['image_bytes'])
            logger.info(f"Transformed image shape: {transformed_img.shape}")
            print(f"Transformed image shape: {transformed_img.shape}")
            transformed_images.append(transformed_img)
            keys.append(row['key'])

        inputs_array = np.array(transformed_images)
        logger.info(f"Inputs array shape: {inputs_array.shape}")
        print(f"Inputs array shape: {inputs_array.shape}")

        # Convert numpy array to list of lists for each image
        inputs_list = [img.tolist() for img in inputs_array]

        # Create DataFrame from transformed images and keys
        inputs_struct = spark.createDataFrame(list(zip(keys, inputs_list)), schema=["key", "transformed_image_bytes"])
        logger.info(f"create dataframe successfully")
        inputs_struct = inputs_struct.withColumn("predictions", image_predict_udf("transformed_image_bytes"))
        logger.info(f"predict successfully")
        pandas_preds_df = inputs_struct.toPandas()
        for index, row in pandas_preds_df.iterrows():
            preds = np.array(row['predictions']).reshape(len(CLASSES), IMG_DIM, IMG_DIM)
            f_mask = reverse_transform_mask(preds)

            # Convert mask to bytes
            mask_bytes = io.BytesIO()
            np.save(mask_bytes, f_mask)
            mask_bytes.seek(0)

            # Save the mask bytes to GridFS
            mask_id = fs.put(mask_bytes, filename=f"{row['key']}_mask.npy")

            # Save the mask ID and image ID to MongoDB
            image_doc = {
                "image_id": row['key'],
                "mask_gridfs_id": mask_id
            }
            collection.insert_one(image_doc)
            logger.info(f"Processed and saved mask for image {row['key']} to MongoDB")
            print(f"Processed and saved mask for image {row['key']} to MongoDB")
    else:
        logger.info("Empty DataFrame received in foreachBatch")
        print("Empty DataFrame received in foreachBatch")

In [11]:

# Clean the checkpoint directory
import shutil
shutil.rmtree("checkpoint_dir", ignore_errors=True)

# Create DataFrame from Kafka stream
streamRawDf = spark.readStream.format("kafka").option("kafka.bootstrap.servers", kafka_server).option("subscribe", topic_name).option("startingOffsets", "latest").load()
streamDF = streamRawDf.select(col("key").cast("string"), col("value").alias("image_bytes"))


In [12]:
# Start the streaming query
query = streamDF.writeStream.foreachBatch(foreach_batch_function).option("checkpointLocation", "checkpoint_dir").start()

# Wait for the termination of the query
query.awaitTermination()


INFO:py4j.java_gateway:Callback Server Starting
INFO:py4j.java_gateway:Socket listening on ('127.0.0.1', 63485)
INFO:py4j.clientserver:Python Server ready to receive messages
INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:Foreach Batch Function called on Epoch ID: 0.


Foreach Batch Function called on Epoch ID: 0.


INFO:__main__:Empty DataFrame received in foreachBatch


Empty DataFrame received in foreachBatch


INFO:py4j.clientserver:Received command c on object id p0
INFO:__main__:Foreach Batch Function called on Epoch ID: 1.


...

INFO:__main__:Processing 10 records in the batch.


Processing 10 records in the batch.


INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Transformed image shape: (786432,)
INFO:__main__:Inputs array shape: (10, 786432)


Transformed image shape: (786432,)
Transformed image shape: (786432,)
Transformed image shape: (786432,)
Transformed image shape: (786432,)
Transformed image shape: (786432,)
Transformed image shape: (786432,)
Transformed image shape: (786432,)
Transformed image shape: (786432,)
Transformed image shape: (786432,)
Transformed image shape: (786432,)
Inputs array shape: (10, 786432)


INFO:__main__:create dataframe successfully
INFO:__main__:predict successfully
